In [1]:
import pandas as pd

yield_rates1 = pd.read_csv('data/yield-curve/yield-curve-rates-1990-2021.csv')
yield_rates2 = pd.read_csv('data/yield-curve/yield-curve-rates-2022.csv')
yield_rates3 = pd.read_csv('data/yield-curve/yield-curve-rates-2023.csv')
yield_rates4 = pd.read_csv('data/yield-curve/yield-curve-rates-2024.csv')

frames = [yield_rates4, yield_rates3, yield_rates2, yield_rates1]
yield_rates = pd.concat(frames, ignore_index=True)
yield_rates

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,01/25/2024,5.54,5.48,5.44,5.39,5.19,4.76,4.28,4.12,4.01,4.07,4.14,4.49,4.38
1,01/24/2024,5.52,5.44,5.44,5.40,5.22,4.83,4.34,4.19,4.06,4.14,4.18,4.52,4.41
2,01/23/2024,5.53,5.46,5.45,5.38,5.21,4.81,4.31,4.16,4.06,4.11,4.14,4.48,4.38
3,01/22/2024,5.53,5.47,5.46,5.39,5.22,4.83,4.37,4.14,4.03,4.07,4.11,4.44,4.32
4,01/19/2024,5.54,5.47,5.45,5.39,5.21,4.84,4.39,4.18,4.08,4.12,4.15,4.47,4.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8519,1/8/90,NaN,NaN,7.79,NaN,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09
8520,1/5/90,NaN,NaN,7.79,NaN,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
8521,1/4/90,NaN,NaN,7.84,NaN,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
8522,1/3/90,NaN,NaN,7.89,NaN,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04


In [2]:
spread_10_3 = []

for i in range(len(yield_rates)):
    spread_10_3.append(yield_rates.loc[i]['10 Yr'] - yield_rates.loc[i]['3 Mo'])

yield_dates = pd.to_datetime(yield_rates['Date'])

new_yield = pd.DataFrame({
    'Date' : yield_dates,
    'spread_10_3' : spread_10_3
})
new_yield

,Date,spread_10_3
0,2024-01-25,-1.30
1,2024-01-24,-1.26
2,2024-01-23,-1.31
3,2024-01-22,-1.35
4,2024-01-19,-1.30
...,...,...
8519,1990-01-08,0.23
8520,1990-01-05,0.20
8521,1990-01-04,0.14
8522,1990-01-03,0.10


Stock Symbols: IYC, IYK, IYE, IYF, IYH, IYJ, IYM, IYW, IYZ, IDU, IYR, IXP

want a function that reads the stock symbols, grabs the corresponding csv, and combines it with the new_yield frame

In [3]:
stock_symbols = ['IYC', 'IYK', 'IYE', 'IYF', 'IYH', 'IYJ', 'IYM',
                 'IYW', 'IYZ', 'IYR', 'IXP']

def stock_yield_curve(symbol):
    stock_data = pd.read_csv('data/etfs/{}.csv'.format(symbol))
    stock_dates = pd.to_datetime(stock_data['Date'])

    new_stock = pd.DataFrame({
        'Date' : stock_dates,
        'Adj Close' : stock_data['Adj Close']
    })

    stock_yield = pd.merge(new_yield, new_stock, how = 'inner')
    df = stock_yield.copy().dropna().iloc[::-1]
    df = df.set_index(df['Date'])
    df.to_csv('{}_yield_curve.csv'.format(symbol))

for symbol in stock_symbols:
    stock_yield_curve(symbol)